In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


# Change training size here

In [3]:
TRAINING_SIZE = 60000
DIGITS = 3

REVERSE = True

MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+- '

RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1


In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)


In [5]:
ctable = CharacterTable(chars)


# Data Generation

In [6]:
questions = []
expected = []
seen = set()

print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                            for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    
    # if operator == 1 do adder, if operator == 2 do substractor
    operator = np.random.choice(list('12'))
    if operator == '1' :
        
        # Pad the data with spaces such that it is always MAXLEN.
        q = '{}+{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        
        ans = str(a + b)
        # Answers can be of maximum size DIGITS + 1.
        ans += ' ' * (DIGITS + 1 - len(ans))
        
    elif operator == '2' :
        # Pad the data with spaces such that it is always MAXLEN.
        q = '{}-{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        
        ans = str(a - b)
        # Answers can be of maximum size DIGITS + 1.
        ans += ' ' * (DIGITS + 1 - len(ans))

    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))



Generating data...
Total addition questions: 60000


In [7]:
print(questions[:10], '\n' ,expected[:10])


['808-3  ', '754-1  ', '32-83  ', '92+50  ', '2+52   ', '8+8    ', '81-0   ', '63+14  ', '12+927 ', '6+9    '] 
 ['805 ', '753 ', '-51 ', '142 ', '54  ', '16  ', '81  ', '77  ', '939 ', '15  ']


# Processing

In [8]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)


Vectorization...


In [9]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:50000]
train_y = y[:50000]
test_x = x[50000:]
test_y = y[50000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)



Training Data:
(45000, 7, 13)
(45000, 4, 13)
Validation Data:
(5000, 7, 13)
(5000, 4, 13)
Testing Data:
(10000, 7, 13)
(10000, 4, 13)


In [10]:
#print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])


# Build Model

In [11]:
print('Build model...')

############################################
##### Build your own model here ############
model = Sequential()

# "Encode" the input sequence using an RNN, input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))

# the decoder RNN's input
model.add(layers.RepeatVector(DIGITS + 1))

# The decoder RNN  with a single layer.
for _ in range(LAYERS):
    # all the outputs in the form of (num_samples, timesteps, output_dim). 
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))


model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))

############################################
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()


Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 13)             1677      
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


# Training

In [12]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # visualizing validation set prediction errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)



--------------------------------------------------
Iteration 0
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 62s 1ms/step - loss: 1.9697 - acc: 0.3051 - val_loss: 1.7997 - val_acc: 0.3483
Q  12+453 T 375  ☒ 100 
Q   2-089 T 978  ☒ 34  
Q   279-7 T -965 ☒ -839
Q 347+718 T 1560 ☒ 100 
Q  42-435 T 510  ☒ 33  
Q  81+354 T 471  ☒ 100 
Q   43+24 T 76   ☒ 10  
Q 561+635 T 701  ☒ 100 
Q  31-136 T 618  ☒ 32  
Q  65+993 T 455  ☒ 100 

--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 26s 570us/step - loss: 1.7589 - acc: 0.3533 - val_loss: 1.7139 - val_acc: 0.3619
Q  76-472 T 207  ☒ 33  
Q 336+765 T 1200 ☒ 104 
Q  251-83 T -114 ☒ -444
Q  75+035 T 587  ☒ 144 
Q   24-78 T 45   ☒ 21  
Q   72+44 T 71   ☒ 54  
Q   822+7 T 235  ☒ 61  
Q 023-405 T 184  ☒ 314 
Q  328-49 T -729 ☒ -584
Q   034-7 T -423 ☒ -544

-------------------

45000/45000 [==============================] - 23s 522us/step - loss: 1.0834 - acc: 0.5954 - val_loss: 1.0721 - val_acc: 0.5977
Q  61-355 T 537  ☒ 521 
Q 617+826 T 1344 ☒ 1345
Q   34-97 T 36   ☒ 35  
Q  97-717 T 638  ☒ 626 
Q  23+296 T 724  ☒ 799 
Q  02-557 T 735  ☒ 721 
Q  72+677 T 803  ☒ 811 
Q   182-2 T -279 ☒ -292
Q  326+41 T 637  ☒ 635 
Q 394+733 T 830  ☒ 850 

--------------------------------------------------
Iteration 15
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 24s 528us/step - loss: 1.0515 - acc: 0.6077 - val_loss: 1.0578 - val_acc: 0.6021
Q  31+449 T 957  ☒ 958 
Q    23-5 T -27  ☑ -27 
Q  32+348 T 866  ☒ 869 
Q 517+917 T 1434 ☒ 1449
Q   2+305 T 505  ☒ 500 
Q 284+349 T 1425 ☒ 1409
Q  639-88 T -848 ☒ -855
Q  467-32 T -741 ☑ -741
Q  83+351 T 191  ☒ 289 
Q   9+819 T 927  ☒ 919 

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 13s 292us/step - loss: 0.1044 - acc: 0.9695 - val_loss: 0.2986 - val_acc: 0.9046
Q 139-201 T -829 ☑ -829
Q 207-499 T 292  ☑ 292 
Q 764-394 T 26   ☒ 12  
Q 726+379 T 1600 ☒ 1501
Q  52+055 T 575  ☑ 575 
Q  59+833 T 433  ☑ 433 
Q 642-009 T 654  ☒ 645 
Q 097-435 T -256 ☑ -256
Q  197-78 T -704 ☑ -704
Q  23-035 T 498  ☑ 498 

--------------------------------------------------
Iteration 62
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 302us/step - loss: 0.1040 - acc: 0.9689 - val_loss: 0.1317 - val_acc: 0.9559
Q  934+08 T 519  ☑ 519 
Q 842-645 T 298  ☒ 208 
Q   4+057 T 754  ☑ 754 
Q  81+513 T 333  ☑ 333 
Q  83-188 T 843  ☑ 843 
Q  251+41 T 166  ☒ 165 
Q 604+245 T 948  ☑ 948 
Q  313-97 T -234 ☑ -234
Q  01-995 T 589  ☑ 589 
Q  19-247 T 651  ☒ 661 

--------------------------------------------------
Iteration 63
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

Q   474-6 T -468 ☑ -468
Q  081+71 T 197  ☑ 197 
Q 127+884 T 1209 ☑ 1209
Q  57-316 T 538  ☑ 538 
Q 862-461 T -104 ☑ -104
Q    1+63 T 37   ☑ 37  
Q 953+588 T 1244 ☑ 1244
Q   594-5 T -490 ☑ -490
Q  26-475 T 512  ☑ 512 
Q   599+4 T 999  ☒ 199 

--------------------------------------------------
Iteration 95
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 172us/step - loss: 0.0245 - acc: 0.9955 - val_loss: 0.0784 - val_acc: 0.9737
Q  35-511 T 62   ☑ 62  
Q  92+707 T 736  ☑ 736 
Q   92-92 T 0    ☒ -   
Q 416-497 T 180  ☑ 180 
Q  01-169 T 951  ☑ 951 
Q  072-27 T -198 ☑ -198
Q 788-784 T -400 ☑ -400
Q  519+16 T 976  ☑ 976 
Q  321-97 T -44  ☑ -44 
Q 227-428 T 102  ☑ 102 

--------------------------------------------------
Iteration 96
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 171us/step - loss: 0.0186 - acc: 0.9973 - val_loss: 0.0730 - val_acc: 0.9759
Q 626-096 T 64   ☑

# Testing

In [13]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 

preds = model.predict_classes(test_x, verbose=0)

i = 0
count = 0
while i < len(preds) :
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)

    print('Q', q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        count += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
    i += 1
    
testing_acc = (count/len(test_y))*100
print('-'*60)
print('-'*30, colors.ok + str(testing_acc) + "%" + colors.close, '-'*30)
print('-'*60)
#####################################################
    

MSG : Prediction
Q 718-122 T 596  ☑ 596 
Q 0-266   T -266 ☑ -266
Q 667+147 T 814  ☑ 814 
Q 879-0   T 879  ☑ 879 
Q 549+544 T 1093 ☑ 1093
Q 656+289 T 945  ☑ 945 
Q 95-396  T -301 ☑ -301
Q 51-36   T 15   ☑ 15  
Q 928+82  T 1010 ☑ 1010
Q 25+552  T 577  ☑ 577 
Q 40-943  T -903 ☑ -903
Q 940+716 T 1656 ☑ 1656
Q 900-825 T 75   ☒ 76  
Q 714-238 T 476  ☑ 476 
Q 881-79  T 802  ☒ 812 
Q 30+623  T 653  ☑ 653 
Q 86+299  T 385  ☑ 385 
Q 373-155 T 218  ☑ 218 
Q 927-855 T 72   ☒ 82  
Q 306-24  T 282  ☑ 282 
Q 618-5   T 613  ☑ 613 
Q 411+161 T 572  ☑ 572 
Q 31-871  T -840 ☑ -840
Q 0-469   T -469 ☑ -469
Q 10+567  T 577  ☑ 577 
Q 740+98  T 838  ☑ 838 
Q 30-771  T -741 ☑ -741
Q 463+775 T 1238 ☑ 1238
Q 411-342 T 69   ☒ 60  
Q 65+544  T 609  ☑ 609 
Q 330-69  T 261  ☑ 261 
Q 2+450   T 452  ☑ 452 
Q 996+694 T 1690 ☑ 1690
Q 819+49  T 868  ☑ 868 
Q 875+7   T 882  ☑ 882 
Q 7+321   T 328  ☑ 328 
Q 160+657 T 817  ☑ 817 
Q 699-634 T 65   ☒ 64  
Q 897+86  T 983  ☑ 983 
Q 18-757  T -739 ☑ -739
Q 8+202   T 210  ☑ 210 

Q 540-70  T 470  ☑ 470 
Q 55-313  T -258 ☑ -258
Q 538-41  T 497  ☑ 497 
Q 609-6   T 603  ☑ 603 
Q 761+466 T 1227 ☑ 1227
Q 83-274  T -191 ☑ -191
Q 256+650 T 906  ☑ 906 
Q 51-234  T -183 ☑ -183
Q 35-932  T -897 ☑ -897
Q 804-983 T -179 ☑ -179
Q 927+271 T 1198 ☑ 1198
Q 703-168 T 535  ☑ 535 
Q 32+906  T 938  ☑ 938 
Q 803-15  T 788  ☑ 788 
Q 134-169 T -35  ☒ -66 
Q 52+546  T 598  ☑ 598 
Q 8-120   T -112 ☑ -112
Q 727+772 T 1499 ☑ 1499
Q 266+8   T 274  ☑ 274 
Q 307-532 T -225 ☑ -225
Q 993-440 T 553  ☒ 543 
Q 230-94  T 136  ☑ 136 
Q 558-24  T 534  ☑ 534 
Q 996-895 T 101  ☒ 1010
Q 875-183 T 692  ☑ 692 
Q 65+597  T 662  ☑ 662 
Q 165+957 T 1122 ☑ 1122
Q 590-546 T 44   ☑ 44  
Q 225-6   T 219  ☑ 219 
Q 418-901 T -483 ☒ -493
Q 242-6   T 236  ☑ 236 
Q 1-850   T -849 ☑ -849
Q 502+532 T 1034 ☑ 1034
Q 462-945 T -483 ☑ -483
Q 2+9     T 11   ☑ 11  
Q 9-214   T -205 ☑ -205
Q 7-90    T -83  ☒ -82 
Q 54+398  T 452  ☑ 452 
Q 77+442  T 519  ☑ 519 
Q 946-733 T 213  ☒ 223 
Q 906+9   T 915  ☑ 915 
Q 9-248   T -239

Q 36-989  T -953 ☑ -953
Q 948-398 T 550  ☑ 550 
Q 68+304  T 372  ☑ 372 
Q 47+327  T 374  ☑ 374 
Q 7-63    T -56  ☑ -56 
Q 771-296 T 475  ☑ 475 
Q 84+975  T 1059 ☑ 1059
Q 688-1   T 687  ☑ 687 
Q 64-215  T -151 ☑ -151
Q 18+921  T 939  ☑ 939 
Q 29-502  T -473 ☑ -473
Q 18+20   T 38   ☑ 38  
Q 295+32  T 327  ☑ 327 
Q 360-348 T 12   ☑ 12  
Q 96-982  T -886 ☒ -876
Q 79-311  T -232 ☑ -232
Q 249+99  T 348  ☑ 348 
Q 934+812 T 1746 ☑ 1746
Q 822-20  T 802  ☑ 802 
Q 33+78   T 111  ☑ 111 
Q 332+636 T 968  ☑ 968 
Q 25-643  T -618 ☑ -618
Q 704+47  T 751  ☑ 751 
Q 15-204  T -189 ☑ -189
Q 39+450  T 489  ☒ 499 
Q 269-7   T 262  ☑ 262 
Q 453-993 T -540 ☒ -439
Q 71+919  T 990  ☑ 990 
Q 680-911 T -231 ☒ -221
Q 289-43  T 246  ☑ 246 
Q 34+221  T 255  ☑ 255 
Q 248+11  T 259  ☑ 259 
Q 438+382 T 820  ☑ 820 
Q 684+36  T 720  ☑ 720 
Q 771+84  T 855  ☑ 855 
Q 808-44  T 764  ☑ 764 
Q 249-85  T 164  ☑ 164 
Q 596+79  T 675  ☑ 675 
Q 72+238  T 310  ☑ 310 
Q 564+13  T 577  ☑ 577 
Q 106+81  T 187  ☑ 187 
Q 204-886 T -682

Q 58-295  T -237 ☑ -237
Q 196+456 T 652  ☑ 652 
Q 787-872 T -85  ☒ -75 
Q 847+12  T 859  ☑ 859 
Q 59-407  T -348 ☑ -348
Q 661+540 T 1201 ☑ 1201
Q 774+55  T 829  ☑ 829 
Q 864+234 T 1098 ☑ 1098
Q 929-24  T 905  ☑ 905 
Q 886-81  T 805  ☑ 805 
Q 81-53   T 28   ☑ 28  
Q 896+33  T 929  ☑ 929 
Q 314+62  T 376  ☑ 376 
Q 775-40  T 735  ☑ 735 
Q 40-204  T -164 ☑ -164
Q 85-481  T -396 ☑ -396
Q 359+20  T 379  ☑ 379 
Q 473-558 T -85  ☑ -85 
Q 7+519   T 526  ☑ 526 
Q 564+23  T 587  ☑ 587 
Q 839+996 T 1835 ☑ 1835
Q 965+785 T 1750 ☑ 1750
Q 632-31  T 601  ☑ 601 
Q 351-947 T -596 ☑ -596
Q 648+123 T 771  ☑ 771 
Q 869-358 T 511  ☑ 511 
Q 661-959 T -298 ☑ -298
Q 189-71  T 118  ☑ 118 
Q 818+153 T 971  ☑ 971 
Q 60-874  T -814 ☑ -814
Q 2-422   T -420 ☑ -420
Q 394+560 T 954  ☑ 954 
Q 102-123 T -21  ☒ -22 
Q 631+961 T 1592 ☑ 1592
Q 769-257 T 512  ☑ 512 
Q 2-55    T -53  ☑ -53 
Q 119+95  T 214  ☑ 214 
Q 396-666 T -270 ☒ -260
Q 79-531  T -452 ☑ -452
Q 12+132  T 144  ☑ 144 
Q 479-4   T 475  ☑ 475 
Q 176+90  T 266 

Q 51-688  T -637 ☑ -637
Q 7-824   T -817 ☑ -817
Q 41+991  T 1032 ☒ 1042
Q 31+201  T 232  ☑ 232 
Q 24-678  T -654 ☑ -654
Q 96-38   T 58   ☑ 58  
Q 846-640 T 206  ☑ 206 
Q 50+528  T 578  ☑ 578 
Q 40-575  T -535 ☑ -535
Q 70-296  T -226 ☑ -226
Q 91-363  T -272 ☑ -272
Q 721-751 T -30  ☑ -30 
Q 547-911 T -364 ☒ -354
Q 689+24  T 713  ☑ 713 
Q 183+244 T 427  ☑ 427 
Q 589+45  T 634  ☑ 634 
Q 453-45  T 408  ☑ 408 
Q 53-58   T -5   ☑ -5  
Q 17+94   T 111  ☑ 111 
Q 944-88  T 856  ☑ 856 
Q 485+914 T 1399 ☑ 1399
Q 293-744 T -451 ☑ -451
Q 452-66  T 386  ☑ 386 
Q 295-11  T 284  ☒ 285 
Q 727+803 T 1530 ☑ 1530
Q 126+680 T 806  ☑ 806 
Q 249+97  T 346  ☑ 346 
Q 24-516  T -492 ☑ -492
Q 62+985  T 1047 ☑ 1047
Q 433-58  T 375  ☑ 375 
Q 425+793 T 1218 ☑ 1218
Q 631-847 T -216 ☑ -216
Q 204+556 T 760  ☑ 760 
Q 741+94  T 835  ☑ 835 
Q 879-874 T 5    ☒ 85  
Q 159+43  T 202  ☑ 202 
Q 12-12   T 0    ☑ 0   
Q 12-616  T -604 ☑ -604
Q 546+98  T 644  ☑ 644 
Q 615-0   T 615  ☑ 615 
Q 64-728  T -664 ☑ -664
Q 392-579 T -187

...

Q 412+683 T 1095 ☑ 1095
Q 90+166  T 256  ☑ 256 
Q 902-279 T 623  ☑ 623 
Q 723+55  T 778  ☑ 778 
Q 37+7    T 44   ☑ 44  
Q 644+62  T 706  ☑ 706 
Q 76-180  T -104 ☑ -104
Q 792-669 T 123  ☑ 123 
Q 857+41  T 898  ☑ 898 
Q 0+470   T 470  ☑ 470 
Q 110-879 T -769 ☑ -769
Q 441+92  T 533  ☑ 533 
Q 856-727 T 129  ☑ 129 
Q 60-397  T -337 ☑ -337
Q 986-19  T 967  ☑ 967 
Q 41+371  T 412  ☑ 412 
Q 15-789  T -774 ☑ -774
Q 403+103 T 506  ☑ 506 
Q 371-603 T -232 ☒ -222
Q 6+752   T 758  ☑ 758 
Q 42-40   T 2    ☒ 3   
Q 817+23  T 840  ☑ 840 
Q 140+95  T 235  ☑ 235 
Q 133+94  T 227  ☑ 227 
Q 843-935 T -92  ☒ -83 
Q 140-297 T -157 ☒ -147
Q 87+75   T 162  ☑ 162 
Q 17+558  T 575  ☑ 575 
Q 57+473  T 530  ☑ 530 
Q 147-63  T 84   ☑ 84  
Q 160+34  T 194  ☑ 194 
Q 32+605  T 637  ☑ 637 
Q 685-599 T 86   ☒ 16  
Q 938+77  T 1015 ☑ 1015
Q 75-271  T -196 ☑ -196
Q 795-623 T 172  ☑ 172 
Q 32-62   T -30  ☑ -30 
Q 4+112   T 116  ☑ 116 
Q 33-332  T -299 ☑ -299
Q 397+31  T 428  ☑ 428 
Q 198+86  T 284  ☑ 284 
Q 938+324 T 1262

Q 741-651 T 90   ☑ 90  
Q 907+33  T 940  ☑ 940 
Q 383-568 T -185 ☒ -175
Q 36+189  T 225  ☑ 225 
Q 883+71  T 954  ☑ 954 
Q 576-6   T 570  ☒ 560 
Q 554+36  T 590  ☑ 590 
Q 526+47  T 573  ☑ 573 
Q 79+88   T 167  ☑ 167 
Q 561+30  T 591  ☑ 591 
Q 886-54  T 832  ☑ 832 
Q 734-34  T 700  ☑ 700 
Q 905+7   T 912  ☑ 912 
Q 59+20   T 79   ☑ 79  
Q 2+301   T 303  ☑ 303 
Q 153+214 T 367  ☑ 367 
Q 128-865 T -737 ☑ -737
Q 396+190 T 586  ☑ 586 
Q 12+13   T 25   ☑ 25  
Q 91+344  T 435  ☑ 435 
Q 50+171  T 221  ☑ 221 
Q 576-708 T -132 ☒ -122
Q 509-31  T 478  ☑ 478 
Q 828-94  T 734  ☑ 734 
Q 513+0   T 513  ☑ 513 
Q 812+91  T 903  ☑ 903 
Q 838+57  T 895  ☑ 895 
Q 153+14  T 167  ☑ 167 
Q 625-53  T 572  ☑ 572 
Q 773-488 T 285  ☑ 285 
Q 307-91  T 216  ☑ 216 
Q 35-445  T -410 ☑ -410
Q 822+32  T 854  ☑ 854 
Q 946+6   T 952  ☑ 952 
Q 37+415  T 452  ☑ 452 
Q 29-394  T -365 ☑ -365
Q 199-741 T -542 ☑ -542
Q 12+702  T 714  ☑ 714 
Q 211+65  T 276  ☑ 276 
Q 291-802 T -511 ☑ -511
Q 779+1   T 780  ☑ 780 
Q 31+166  T 197 

Q 662-36  T 626  ☑ 626 
Q 77+8    T 85   ☑ 85  
Q 60+540  T 600  ☑ 600 
Q 553-24  T 529  ☑ 529 
Q 338-90  T 248  ☑ 248 
Q 53-489  T -436 ☑ -436
Q 803+29  T 832  ☑ 832 
Q 935+261 T 1196 ☑ 1196
Q 0-607   T -607 ☑ -607
Q 204-78  T 126  ☑ 126 
Q 184-77  T 107  ☑ 107 
Q 77-85   T -8   ☒ -   
Q 481+710 T 1191 ☒ 1291
Q 498-93  T 405  ☑ 405 
Q 44-823  T -779 ☑ -779
Q 730-80  T 650  ☑ 650 
Q 50+72   T 122  ☑ 122 
Q 241+327 T 568  ☑ 568 
Q 95-78   T 17   ☑ 17  
Q 793-760 T 33   ☒ 24  
Q 59-659  T -600 ☒ -500
Q 531+778 T 1309 ☑ 1309
Q 525-430 T 95   ☒ 94  
Q 360-35  T 325  ☑ 325 
Q 622+316 T 938  ☑ 938 
Q 305-549 T -244 ☒ -234
Q 111+69  T 180  ☑ 180 
Q 50-895  T -845 ☑ -845
Q 629+25  T 654  ☑ 654 
Q 929-641 T 288  ☑ 288 
Q 60+154  T 214  ☑ 214 
Q 99+10   T 109  ☑ 109 
Q 89-653  T -564 ☑ -564
Q 982+85  T 1067 ☑ 1067
Q 28-728  T -700 ☒ -600
Q 685-79  T 606  ☑ 606 
Q 430-236 T 194  ☑ 194 
Q 94+949  T 1043 ☒ 1044
Q 558-735 T -177 ☒ -167
Q 123+175 T 298  ☑ 298 
Q 315-885 T -570 ☒ -579
Q 76+157  T 233 

Q 4+213   T 217  ☑ 217 
Q 373-465 T -92  ☑ -92 
Q 797+599 T 1396 ☑ 1396
Q 23-970  T -947 ☒ -946
Q 94-816  T -722 ☒ -721
Q 54+849  T 903  ☑ 903 
Q 393-808 T -415 ☑ -415
Q 3-968   T -965 ☑ -965
Q 41+870  T 911  ☑ 911 
Q 170-53  T 117  ☑ 117 
Q 984-9   T 975  ☑ 975 
Q 6-220   T -214 ☑ -214
Q 19-3    T 16   ☑ 16  
Q 605-7   T 598  ☑ 598 
Q 814+499 T 1313 ☑ 1313
Q 901+34  T 935  ☑ 935 
Q 743-34  T 709  ☑ 709 
Q 51+708  T 759  ☑ 759 
Q 37-461  T -424 ☑ -424
Q 69+672  T 741  ☑ 741 
Q 5+684   T 689  ☒ 699 
Q 97+9    T 106  ☑ 106 
Q 36+507  T 543  ☑ 543 
Q 319+950 T 1269 ☒ 1259
Q 2+634   T 636  ☑ 636 
Q 26-573  T -547 ☑ -547
Q 602+67  T 669  ☑ 669 
Q 208+84  T 292  ☑ 292 
Q 98+407  T 505  ☑ 505 
Q 801+393 T 1194 ☑ 1194
Q 254-571 T -317 ☑ -317
Q 314-27  T 287  ☑ 287 
Q 236+416 T 652  ☑ 652 
Q 287-298 T -11  ☒ -1  
Q 33-17   T 16   ☑ 16  
Q 882-34  T 848  ☑ 848 
Q 646-900 T -254 ☑ -254
Q 609-374 T 235  ☑ 235 
Q 121+92  T 213  ☑ 213 
Q 655-70  T 585  ☑ 585 
Q 230+449 T 679  ☑ 679 
Q 89-150  T -61 

Q 775+17  T 792  ☑ 792 
Q 7-298   T -291 ☑ -291
Q 961+713 T 1674 ☑ 1674
Q 981+674 T 1655 ☑ 1655
Q 766-349 T 417  ☑ 417 
Q 419-746 T -327 ☑ -327
Q 83-905  T -822 ☑ -822
Q 24+585  T 609  ☑ 609 
Q 45+68   T 113  ☑ 113 
Q 64+547  T 611  ☑ 611 
Q 515-38  T 477  ☑ 477 
Q 5+421   T 426  ☑ 426 
Q 57-624  T -567 ☑ -567
Q 8+67    T 75   ☑ 75  
Q 25-366  T -341 ☑ -341
Q 861+58  T 919  ☑ 919 
Q 59+104  T 163  ☑ 163 
Q 40-172  T -132 ☑ -132
Q 981+201 T 1182 ☑ 1182
Q 331+927 T 1258 ☑ 1258
Q 9-232   T -223 ☑ -223
Q 53+186  T 239  ☒ 249 
Q 0+593   T 593  ☑ 593 
Q 17+487  T 504  ☑ 504 
Q 167-6   T 161  ☑ 161 
Q 260-7   T 253  ☑ 253 
Q 106-1   T 105  ☑ 105 
Q 11-695  T -684 ☑ -684
Q 80+0    T 80   ☒ 70  
Q 969-315 T 654  ☒ 644 
Q 330+813 T 1143 ☒ 1243
Q 514-88  T 426  ☑ 426 
Q 147-806 T -659 ☑ -659
Q 187-56  T 131  ☑ 131 
Q 622-99  T 523  ☑ 523 
Q 0+86    T 86   ☑ 86  
Q 510+96  T 606  ☑ 606 
Q 836-83  T 753  ☑ 753 
Q 679+91  T 770  ☒ 760 
Q 4+455   T 459  ☑ 459 
Q 9-485   T -476 ☑ -476
Q 54-293  T -239